# LIXO 
---

OCUPANTES

In [ ]:
async def run(self):
            '''
            se saida-> vai para saida
            se não random
            '''

            if self.clossest_exit()!= -1:
                EvacuationUI.update_agent_position(self.agent.name,self.clossest_exit())

            if self.agent.exits["Exit 1"] == 'closed':
                print(f"{self.agent.name} found Exit 1 clsoed, rerouting to another exit...")

            else:
                print(f"{self.agent.name} is heading to Exit 1")

            if self.agent.elevator == 'off':
                print(f"{self.agent.name} found the elevator off, will try to use stairs")


            await asyncio.sleep(2)

In [ ]:
async def clossest_exit(self):
            
            #get poss of ocupant based on name / name==poss na list
            poss_exits = EvacuationUI.exits_doors()

            ###ver caso não existam saídas(saida==curr_poss ou -1)-> andar aleatóriamente
            if not poss_exits:return -1

            x, y = EvacuationUI.occupants_loc[self.name] #[x, y]

            #ver prioridade -> mais á frente... :)

            dist = EvacuationUI.grid_size + 1
            i = 0
            p_dist = -1

            for i in range(poss_exits):
                x1 = poss_exits[i][0]
                y1 = poss_exits[i][1]
                if np.sqrt((x-x1)**2 + (y-y1)**2)<dist:
                    dist = np.sqrt((x-x1)**2 + (y-y1)**2)
                    p_dist = i

            return poss_exits[p_dist][0], poss_exits[p_dist][1]

        def find_path(self, target):
            
            
            open_set = []
            position = self.environment.get_occupant_loc(self.jid)
            heapq.heappush(open_set, (0, position))  # Priority queue with (cost, position)
            came_from = {}
            g_score = {position: 0}
            f_score = {position: self.heuristic(position, target)}

            while open_set:
                _, current = heapq.heappop(open_set)

                # Exit reached
                if current == target:  
                    #o occ guarda o caminho q quer fazer 
                    self.path = self.reconstruct_path(came_from, current)
                    return self.path



                x, y = current
                neighbors = self.astar_possible_moves(x, y) #where he can walk to(they are poss moves)
                for nx, ny in neighbors:
                    tentative_g_score = g_score[current] + 1
                    if tentative_g_score < g_score.get((nx, ny), float('inf')):
                        came_from[(nx, ny)] = current
                        g_score[(nx, ny)] = tentative_g_score
                        f_score[(nx, ny)] = tentative_g_score + self.heuristic((nx, ny), target)

                        # Avoid duplicates in the open_set
                        if (nx, ny) not in f_score:
                            heapq.heappush(open_set, (f_score[(nx, ny)], (nx, ny)))

            return None  # No path found

        def heuristic(self, a, b):
            
            return np.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

        def reconstruct_path(self, came_from, current):
            
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.reverse()
            return path


---

BUILDING_AGENT

----

EMERGENCY_RESPONDER

---